# Paging

We want to be dynamically allocate memory, but also ensure that our physical memory doesn't become fragmented.

The basic idea is that we break physical memory is equal size chunks which give out as needed.

We have two memory spaces to keep in mind, the Virtual Address Space and the Physical memory.

We break the Virtual Address Space (VAS) into **Virtual Pages**.

For example, an 64 byte address space:

<br>
<img src="images/07-paging-VAS.png" width="500">
<br>

We break physical memory into **Physical Frames**.

The OS maps virtual pages to physical frames.

`VP_0` has to be in some physical frame in memory.

Virtual pages and physical frames must be of the same size.

<br>
<img src="images/08-paging-phys-mem.png" width="500">
<br>

The OS stores the mapping from virtual pages to physical frames as an array referred to as the **Page Table**.

This is a per-process data structure.

The 0th index of the page table tells us where `VP-0` is stored at in physical memory.

The 1st index of the page table tells us where `VP-1` is stored at in physical memory.

The Page Table is stored in OS memory. There is a specific register which holds the memory address of the beginning of the Page Table array.

This register is called the **Page Table Base Register (PTBR)**

# The Anatomy of a Virtual Address

Each VA must contain:

1) which virtual page this address is referencing: the Virtual Page Number (VPN)
2) how far into that virtual page we want to access: the OFFSET

<br>
<img src="images/09-paging-VA.png" width="300">
<br>

With a 64 byte address space, we need 6 bits to distinctly refer to every byte.

With 4 total pages, we need 2 bits to serve as the VPN, leaving 4 bits as the OFFSET.

For the above example, the VPN is 1, and we want the 5th byte in Virtual Page 0

Another Example:

```
VA: 111010
VA: 11 | 1010
   VPN | OFFSET
VPN: 3
OFFSET: 10

We want the 10th byte in the `VP_3`.

# Address Translation

When a process requests access to some Virtual Address, the OS:

1) breaks the VA into its parts: the VPN and OFFSET
2) looks up in the Page Table which Physical Frame this Virtual Page is stored on
3) calculates the physical address by concatenating the OFFSET to the Physical Frame Number

<br>
<img src="images/10-VA-to-PA.png" width="400">
<br>

<br>
<img src="images/11-phys-mem.png" width="500">
<br>

# The Page Table

The Page Table is an array of Page Table Entries (PTE).

Every PTE contains the Physical Frame Number that this page maps to and extra bits of information such:

- is this frame valid (has it been allocated to this process yet)
- is it protected (kernel mode needed to acces it)
- is it in RAM or has it been swapped to disk
- ...

<br>
<img src="images/12-PTE.png" width="500">
<br>

To perform an address translation:

<br>
<img src="images/13-paging-mem-access.png" width="500">
<br>

Problem with the above:

It is TOO SLOW. Every memory access requires two memory accesses. 

## Virtual Addresses in Practice

In practice a reasonable size is a 32-bit virtual address.

A good size for a page is 4KB: 4096 bytes

We need 12 bits in order to access into pages of size 4096 because

4KB = 1KB  * 4
    = 1024 * 4
    = 2^10 * 2^2
    = 2^12

If we have a 32 bit address and use 12 of them for the OFFSET, then that leaves 20 bits for the VPN.

```
|               VPN             |  OFFSET |
32                              11        0
```